In [22]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content/'
!kaggle competitions download -c dogs-vs-cats-redux-kernels-edition
!unzip -q train.zip -d . 

100% 543M/544M [00:04<00:00, 169MB/s]
100% 544M/544M [00:04<00:00, 127MB/s]
 95% 258M/271M [00:02<00:00, 164MB/s]
100% 271M/271M [00:02<00:00, 133MB/s]
  0% 0.00/111k [00:00<?, ?B/s]
100% 111k/111k [00:00<00:00, 112MB/s]


In [23]:
%rm -rf dataset

In [24]:
import os 
import tensorflow as tf 
import shutil

os.mkdir('/content/dataset')
os.mkdir('/content/dataset/cat')
os.mkdir('/content/dataset/dog')

for i in os.listdir('/content/train/'):
  if 'cat' in i:
    shutil.copyfile( '/content/train/' + i , '/content/dataset/cat/' + i )
  if 'dog' in i:
    shutil.copyfile( '/content/train/' + i , '/content/dataset/dog/' + i )

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  '/content/dataset/',
  image_size=(150,150),
  batch_size=64,
  subset='training',
  validation_split=0.2,
  seed=1234
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  '/content/dataset/',
  image_size=(150,150),
  batch_size=64,
  subset='validation',
  validation_split=0.2,
  seed=1234
)

print(train_ds)

def 전처리함수(i, 정답):
  i = tf.cast( i/255.0, tf.float32 )
  return i, 정답

train_ds = train_ds.map(전처리함수)
val_ds = val_ds.map(전처리함수)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
<BatchDataset shapes: ((None, 150, 150, 3), (None,)), types: (tf.float32, tf.int32)>


In [ ]:
import requests

url = 'https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
r = requests.get(url, allow_redirects=True)

open('inception_v3.h5', 'wb').write(r.content)

87910968

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

inception_model  = InceptionV3( input_shape=(150,150,3),include_top=False, weights=None)#Dense모델은 사용하지 않겠다(마지막 출력층),
inception_model.load_weights('inception_v3.h5')
##불러오기종료
# inception_model.summary()
#분석
#마지막 레이어들은 제외한 레이어들은 Conv+Pooling으로 이미지 해체분석을 해줍니다.
#그래서 결론은 Dense레이어만 업데이트를 진행하면 됩니다.

for i in inception_model.layers:
  i.trainable= False  #W 값 고정 트레이닝을 안하겠다.
  
마지막레이어=inception_model.get_layer('mixed7')


In [27]:
import tensorflow as tf

layer1 = tf.keras.layers.Flatten()(마지막레이어.output)
layer2 = tf.keras.layers.Dense(1024, activation='relu')(layer1)
drop1 = tf.keras.layers.Dropout(0.2)(layer2)
layer3 = tf.keras.layers.Dense(1, activation='sigmoid')(drop1)

model= tf.keras.Model(inception_model.input,layer3)

model.compile( loss='binary_crossentropy' , optimizer='adam', metrics=['acc'])
model.fit(train_ds,validation_data=val_ds, epochs=2)

Epoch 1/2
313/313 [==============================] - 912s 3s/step - loss: 0.1567 - acc: 0.9552 - val_loss: 0.1018 - val_acc: 0.9576
Epoch 2/2
313/313 [==============================] - 912s 3s/step - loss: 0.0409 - acc: 0.9845 - val_loss: 0.1058 - val_acc: 0.9668
